In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet

In [157]:
df1=pd.read_parquet('bike-201801.parquet')
df2=pd.read_parquet('bike-201802.parquet')
df3=pd.read_parquet('bike-201803.parquet')
df4=pd.read_parquet('bike-201804.parquet')
df5=pd.read_parquet('bike-201805.parquet')
df6=pd.read_parquet('bike-201806.parquet')
df7=pd.read_parquet('bike-201807.parquet')
df8=pd.read_parquet('bike-201808.parquet')
df9=pd.read_parquet('bike-201809.parquet')
df10=pd.read_parquet('bike-201810.parquet')
df11=pd.read_parquet('bike-201811.parquet')
df12=pd.read_parquet('bike-201812.parquet')


In [118]:
df1.shape

(718994, 15)

In [182]:
sample_size = int(len(df12) / 2)
random_sample = df11.sample(n=sample_size)
random_sample.shape

(508252, 15)

In [183]:
random_sample.to_parquet('201812')

In [109]:
df1.shape

(718994, 15)

In [184]:
df1=pd.read_parquet('201801')
df2=pd.read_parquet('201802')
df3=pd.read_parquet('201803')
df4=pd.read_parquet('201804')
df5=pd.read_parquet('201805')
df6=pd.read_parquet('201806')
df7=pd.read_parquet('201807')
df8=pd.read_parquet('201808')
df9=pd.read_parquet('201809')
df10=pd.read_parquet('201810')
df11=pd.read_parquet('201811')
df12=pd.read_parquet('201812')

In [185]:
merge_data=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12],axis=0)

In [186]:
merge_data.to_parquet('bike_data')

In [11]:
df=pd.read_parquet('bike_data')
df.shape

(8774166, 15)

In [3]:
df.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
674060,189,2018-01-25 08:33:09.4370,2018-01-25 08:36:19.2520,3472.0,W 15 St & 10 Ave,40.742754,-74.007474,482.0,W 15 St & 7 Ave,40.739355,-73.999318,26337,Subscriber,1989,1
6387,193,2018-01-30 11:26:52.0600,2018-01-30 11:30:05.2500,127.0,Barrow St & Hudson St,40.731724,-74.006744,346.0,Bank St & Hudson St,40.736529,-74.006180,19854,Subscriber,1954,2


In [4]:
start_column=['starttime','start station id','start station latitude','start station longitude']
df[start_column].to_parquet('start_data')

In [6]:
df=pd.read_parquet('start_data')
df=df.rename(columns={'starttime':'datetime','start station id':'station id'})

In [8]:
merged_column = [(x, y) for x, y in zip(df['start station latitude'], df['start station longitude'])]

df['GeoLocation'] = merged_column

print(df.head(2))


                        datetime  station id  start station latitude  \
674060  2018-01-25 08:33:09.4370      3472.0               40.742754   
6387    2018-01-30 11:26:52.0600       127.0               40.731724   

        start station longitude                              GeoLocation  
674060               -74.007474  (40.74275382865903, -74.00747358798981)  
6387                 -74.006744              (40.73172428, -74.00674436)  


In [9]:
df.to_parquet('clean_start_data')

In [12]:
end_column=['stoptime','end station id','end station latitude','end station longitude']
df[end_column].to_parquet('end_data')

In [14]:
df=pd.read_parquet('end_data')
df=df.rename(columns={'stoptime':'datetime','end station id':'station id'})

In [15]:
merged_column = [(x, y) for x, y in zip(df['end station latitude'], df['end station longitude'])]

df['GeoLocation'] = merged_column

print(df.head(2))


                        datetime  station id  end station latitude  \
674060  2018-01-25 08:36:19.2520       482.0             40.739355   
6387    2018-01-30 11:30:05.2500       346.0             40.736529   

        end station longitude                  GeoLocation  
674060             -73.999318  (40.73935542, -73.99931783)  
6387               -74.006180  (40.73652889, -74.00618026)  


In [16]:
df.to_parquet('clean_end_data')

In [2]:
df1=pd.read_parquet('clean_start_data')
df2=pd.read_parquet('clean_end_data')

In [3]:
df1=df1.drop(['start station latitude','start station longitude'],axis=1)
df1['action']=0
df1.head(2)

,datetime,station id,GeoLocation,action
674060,2018-01-25 08:33:09.4370,3472.0,"[40.74275382865903, -74.00747358798981]",0
6387,2018-01-30 11:26:52.0600,127.0,"[40.73172428, -74.00674436]",0


In [4]:
df2=df2.drop(['end station latitude','end station longitude'],axis=1)
df2['action']=1
df2.head(2)

,datetime,station id,GeoLocation,action
674060,2018-01-25 08:36:19.2520,482.0,"[40.73935542, -73.99931783]",1
6387,2018-01-30 11:30:05.2500,346.0,"[40.73652889, -74.00618026]",1


In [26]:
df2.shape

(8774166, 4)

In [5]:
merged_data=pd.concat([df1,df2],axis=0)

merged_data.to_parquet('bike_merged_data')

In [2]:
df_merged_data=pd.read_parquet('bike_merged_data')


In [7]:
df_merged_data.head(2)

,datetime,station id,GeoLocation,action
674060,2018-01-25 08:33:09.4370,3472.0,"[40.74275382865903, -74.00747358798981]",0
6387,2018-01-30 11:26:52.0600,127.0,"[40.73172428, -74.00674436]",0


In [4]:
df_merged_data['datetime']=pd.to_datetime(df_merged_data['datetime'])

In [5]:
df_merged_data.dtypes

datetime       datetime64[ns]
station id            float64
GeoLocation            object
action                  int64
dtype: object

In [6]:
df_merged_data['datetime_rounded'] = df_merged_data['datetime'].dt.floor('15min')

In [7]:
df_merged_data['interval'] = df_merged_data['datetime_rounded'].dt.strftime('%M')


In [8]:
df_merged_data['interval'] = df_merged_data['interval'].replace('60', '00')

In [9]:
df_merged_data.head(10)

,datetime,station id,GeoLocation,action,datetime_rounded,interval
674060,2018-01-25 08:33:09.437,3472.0,"[40.74275382865903, -74.00747358798981]",0,2018-01-25 08:30:00,30
6387,2018-01-30 11:26:52.060,127.0,"[40.73172428, -74.00674436]",0,2018-01-30 11:15:00,15
75008,2018-01-13 16:10:07.262,280.0,"[40.73331967, -73.99510132]",0,2018-01-13 16:00:00,00
439913,2018-01-11 19:07:06.734,540.0,"[40.74311555376486, -73.98215353488922]",0,2018-01-11 19:00:00,00
523068,2018-01-11 09:02:35.600,3159.0,"[40.77492513, -73.98266566]",0,2018-01-11 09:00:00,00
109920,2018-01-05 16:03:56.810,310.0,"[40.68926942, -73.98912867]",0,2018-01-05 16:00:00,00
471080,2018-01-03 21:41:24.760,3064.0,"[40.69681963, -73.93756926]",0,2018-01-03 21:30:00,30
425593,2018-01-10 11:06:20.837,530.0,"[40.771522, -73.990541]",0,2018-01-10 11:00:00,00
641105,2018-01-15 02:39:31.385,3417.0,"[40.6795766, -73.97854971]",0,2018-01-15 02:30:00,30
70876,2018-01-22 17:16:52.227,270.0,"[40.69308257, -73.97178913]",0,2018-01-22 17:15:00,15


In [10]:
select_column=['datetime_rounded','action','GeoLocation','station id']
df_merged_data[select_column].to_parquet('bike_merged_data')

In [2]:
df=pd.read_parquet('bike_merged_data')

In [4]:
unique_rows = df.drop_duplicates(subset=['station id'])
print(unique_rows.count())


datetime_rounded    839
action              839
GeoLocation         839
station id          838
dtype: int64


In [5]:
selected_columns = ['GeoLocation', 'station id']  # 自定义需要保留的列名
filtered_rows = unique_rows.loc[:, selected_columns]

print(filtered_rows.head(2))


                                    GeoLocation  station id
674060  [40.74275382865903, -74.00747358798981]      3472.0
6387                [40.73172428, -74.00674436]       127.0


In [7]:
filtered_rows.to_csv('stationID_geoloaction.csv')